In [ ]:
# 1. RUN CONFIG: FUNCTIONS, CONSTANTS, MODELS, ETC
%run "../config/config.ipynb"

list_dataset = [ds0]
list_forecast_horizon = [fh1] # 30min ahead

list_model_and_hp = [
    [m1, 'hp1'],
    [m2, 'hp2'],
    [m3, 'hp1'],
    [m4, 'hp1'],
    [m5, 'hp1'],
    [m6, 'hp1'],
    [m7, 'hp1'],
    [m8, 'hp1'],
    [m9, 'hp3'],
    [m10, 'hp1'],
    [m11, 'hp1'],
    [m12, 'hp1'],
    [m13, 'hp1'],
    [m14, 'hp1'],
    [m15, 'hp1'],
    [m16, 'hp1'],
    [m17, 'hp1'],
    [m18, 'hp1']
]

In [4]:
# RUN BATCH FOR ALL COMBINATIONS
for dataset in list_dataset:
    for forecast_horizon in list_forecast_horizon:
        for model_name, hyperparameter_no in list_model_and_hp:
            print(f"Running {model_name} with hyperparameter {hyperparameter_no} on {dataset} for forecast horizon {forecast_horizon}")
            dataset = dataset
            forecast_horizon = forecast_horizon # fh1 = 30 minutes ahead, fh9 = 2 days ahead

            # MODEL AND HYPERPARAMETER TO CHOOSE
            model_name = model_name
            hyperparameter_no = hyperparameter_no

            # 3. PREPARE FOLDER
            hyperparameter, experiment_no_str, filepath = prepare_directory(path_result, forecast_horizon, model_name, hyperparameter_no)

            # 4. INPUT DATA
            block_length, holdout_df, df = input_and_process(path_data_cleaned, forecast_horizon, max_lag_day, n_block)

            # 5. RUN MODEL
            run_model(df, model_name, hyperparameter, filepath, forecast_horizon)

Running m18_nbeats with hyperparameter hp1 on ds0_test.csv for forecast horizon 30
Pass Test 1 - Forecast horizon is >= dataset frequency
Pass Test 2 - Hyperparameter choice is possible given the forecast horizon
Processing CV 1 / 10....

Processing CV 2 / 10....

Processing CV 3 / 10....

Processing CV 4 / 10....

Processing CV 5 / 10....

Processing CV 6 / 10....

Processing CV 7 / 10....

Processing CV 8 / 10....

Processing CV 9 / 10....

Processing CV 10 / 10....



# Populate Last 18 Experiment Cross Validation Results For Testing

In [91]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
from datetime import date

exp_result_folder = '../../experiment_result/'
benchmark_file = '../../experiment_result/Archive/Testing Result/testing_benchmark.csv'
output_folder = '../../experiment_result/Archive/Testing Result/'

# Get m1-m18 name

# Create an empty Df
df = pd.DataFrame()


# Compute the number of folder in the experiment result folder 
def get_number_of_folders(path):
    return len([name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))])

# Get the number of folders in the experiment result folder
number_of_folders = get_number_of_folders(exp_result_folder)
n_experiments = number_of_folders - 1
pos_first_naive = n_experiments - 18 + 1
parent = Path(exp_result_folder)        # adjust

summary_df = pd.DataFrame()

for i in range(pos_first_naive, n_experiments + 1):        # 1 → n (change to 0-based if you need)
    # “E” + 5-digit zero-padded index, e.g. 1 → E00001, 12 → E00012, 123 → E00123
    pattern = f"E{i:05d}_*"                  # tail “_*” matches any suffix
    for folder in parent.glob(pattern):      # returns Path objects for every match
        print('Extracting ' + str(folder))                        # ↪ or os.chdir(folder) / open files here
        
        # Extract model name
        
        import re

        match = re.search(r"(m\d+_[^_]+)", str(folder))
        if match:
            model = match.group(1)
        
        try:
            # any file that ends with “…_cross_validation_result[.csv]”
            csv_path = next(folder.glob("*_cross_validation_result*.csv"))
            result_df = pd.read_csv(csv_path, header=0, skiprows=range(1, 11), nrows=2, index_col=0).T     #  result
            
            result_df = result_df.stack()
            result_df.index = result_df.index.map(lambda x: f"{x[0]}_{x[1]}")
            result_df = result_df.to_frame(name="value, test")
            result_df.index = model + "_" + result_df.index.astype(str)

            summary_df = pd.concat([summary_df, result_df], axis=0)

        except StopIteration:
            print(f"⚠️ no result file in {folder}")
        except Exception as e:
            print(f"⚠️ error reading {folder}: {e}")

# Copy benchmark values
benchmark_df = pd.read_csv(benchmark_file, index_col=0)

summary_df = pd.concat([benchmark_df, summary_df], axis=1)

summary_df['error_percent'] = (
    (summary_df['value, test'] - summary_df['value, benchmark']) 
    / (summary_df['value, benchmark'] + 1e-8) * 100
).round(2)

summary_df['test_result'] = np.where(summary_df['error_percent'] < 5, 'pass', 'fail')

# Create output file name


output_file = f"{date.today().strftime('%Y%m%d')}_test_result.csv"

# Output testing file result
print('Exporting ' + output_file)  # e.g. 20250819_test_result.csv
summary_df.to_csv(output_folder + output_file, index=True)


Extracting ..\..\experiment_result\E00010_250818_ds0_fh30_m1_naive_hp1
Extracting ..\..\experiment_result\E00011_250818_ds0_fh30_m2_snaive_hp2
Extracting ..\..\experiment_result\E00012_250818_ds0_fh30_m3_ets_hp1
Extracting ..\..\experiment_result\E00013_250818_ds0_fh30_m4_arima_hp1
Extracting ..\..\experiment_result\E00014_250818_ds0_fh30_m5_sarima_hp1
Extracting ..\..\experiment_result\E00015_250818_ds0_fh30_m6_lr_hp1
Extracting ..\..\experiment_result\E00016_250818_ds0_fh30_m7_ann_hp1
Extracting ..\..\experiment_result\E00017_250818_ds0_fh30_m8_dnn_hp1
Extracting ..\..\experiment_result\E00018_250818_ds0_fh30_m9_rt_hp3
Extracting ..\..\experiment_result\E00019_250818_ds0_fh30_m10_rf_hp1
Extracting ..\..\experiment_result\E00020_250818_ds0_fh30_m11_svr_hp1
Extracting ..\..\experiment_result\E00021_250818_ds0_fh30_m12_rnn_hp1
Extracting ..\..\experiment_result\E00022_250818_ds0_fh30_m13_lstm_hp1
Extracting ..\..\experiment_result\E00023_250818_ds0_fh30_m14_gru_hp1
Extracting ..\..\expe